## gensimを使ってLSI

以下を実行しておく

```
cd learning

curl https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2 -o ./prototype/workingjawiki-latest-pages-articles.xml.bz2
python bin/generators/wiki_corpus.py ./prototype/workingjawiki-latest-pages-articles.xml.bz2 ./prototype/working/wiki
python bin/generators/lsi.py ./prototype/working/wiki ./prototype/working/
```

num_topicsは1000

In [1]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.similarities import MatrixSimilarity
from gensim.models import LsiModel, TfidfModel

In [2]:
dictionary = Dictionary.load_from_text('../working/wiki_wordids.txt.bz2')
lsi = LsiModel.load('../working/lsi_1000.model')

mm = MmCorpus('../working/wiki_tfidf.mm')

In [3]:
import pandas as pd
data = pd.read_csv('../fixtures/learning_training_messages/LSI_in_sckit_learn.csv')
data

,id,question,answer,question_answer_id,bot_id
0,3186876,育児短時間勤務申請書はいつまでに提出するのか？,1ヵ月前まで（年度申請）,13290,13
1,3186877,育児短時間勤務申請書は曜日別に申請できるか？,1日の勤務時間が6時間を超えていれば問題ありません。,13291,13
2,3186878,育児短時間勤務申請はどこへ提出すればよいか？,各拠店のTHR担当へご提出下さい。,13292,13
3,1,子供が生まれた,子供が生まれたのですね,1,13
4,2,VISAの勘定科目がわからない,VISAの勘定科目がわからないのですね,2,13
5,3,海外出張したときの旅費精算をしたいがどうしたら良い,海外出張の旅費精算したいんですね,3,13
6,4,保険証を失くした,保険証を失くしたのですね,4,13


In [4]:
from tokenizer import MecabTokenizer
questions = MecabTokenizer().tokenize(data['question'])
questions[:3]

[['育児', '短時間', '勤務', '申請', '書', 'いつ', '提出', 'する'],
 ['育児', '短時間', '勤務', '申請', '書', '曜日', '別', '申請', 'できる'],
 ['育児', '短時間', '勤務', '申請', 'どこ', '提出', 'する', 'よい']]

In [5]:
tfidf = TfidfModel.load('../working/wiki.tfidf_model')
id_corpus=[dictionary.doc2bow(sentence, allow_update=True) for sentence in questions]
tfidf_corpus = tfidf[id_corpus]
lsi_corpus = lsi[tfidf_corpus]
lsi_corpus

In [6]:
question = MecabTokenizer().tokenize(['保険証なくした'])
print(question)
id_q_corpus = dictionary.doc2bow(question[0], allow_update=True)
lsi_q_corpus = lsi[tfidf[id_q_corpus]]
lsi_q_corpus[:3]

[['保険証', 'ない', 'する']]


[(0, 0.00011094020706942086),
 (1, 3.0193065867339513e-05),
 (2, 9.6321193365735345e-05)]

In [7]:
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity

lsi_index = SparseMatrixSimilarity(lsi_corpus, num_features=1000)

In [8]:
sims = lsi_index[lsi_q_corpus]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

[(6, 1.0000001),
 (0, 0.13569847),
 (2, 0.1306518),
 (1, 0.11313108),
 (5, 0.11034351),
 (4, 0.012417819),
 (3, -0.016704276)]